<h3>Fetching The Data from Twitter API</h3>

In [1]:
import pandas as pd
import numpy as np
import tweepy
import re

In [2]:
#Twitter API Keys
consumerKey = "5aUYLw0kl8aug0ReGxtdPabZN"
consumerSecret = "aJHj7FMeA5adpZC1Ixbq2X7ObW0qOJx4e7URYX2WgA0j76fDFJ"
accessToken = "1384610198078230531-UG3OwzfuXUmcCnY4mKsIdkTlTvY63A"
accessTokenSecret = "for1lHlL1xU8Gz1bHuUHXanYKWdS3MnbX1liHUToRrdkR"

In [3]:
#Create the authentication object
authenticate = tweepy.OAuthHandler(consumerKey, consumerSecret)
#Set the access token and access token secret
authenticate.set_access_token(accessToken, accessTokenSecret)
#Create the API object while passing the auth information
api = tweepy.API(authenticate, wait_on_rate_limit = True)

In [5]:
tweets_df[0:10]

,tweet_text,tweet_datetime,tweet_Id,Twitter_name
0,RT @TaraBull808: @CEOAdam I'd love to invite y...,2021-05-07 01:22:07,1390477009751482376,jamiegagnon
1,RT @Dogecoinrich: What is your #Dogecoin price...,2021-05-07 01:22:07,1390477008694431746,JustStocksToday
2,"Come check us out, live right now! \n\nWe LOVE...",2021-05-07 01:22:07,1390477007713091584,CHRISsW0RLD
3,@elonmusk How quickly can we destroy the moon ...,2021-05-07 01:22:07,1390477007469780994,TKO_RS_One_
4,RT @MattWallace888: Elon Musk Reveals Mars Dat...,2021-05-07 01:22:05,1390477000658276356,OMEGAoooFIRE
5,RT @MattWallace888: @nbcsnl The SNL Dogecoin s...,2021-05-07 01:22:05,1390477000515768322,Moe67378043
6,RT @Indians: Dogecoin and Franmil home runs ar...,2021-05-07 01:22:05,1390477000138235914,A_Bauer209
7,RT @ItsDogeCoin: I’m HODLing. Retweet &amp; pa...,2021-05-07 01:22:04,1390476998758309890,itwascytronical
8,@FinanceLokum Very Nice Project \n\n@Natasha38...,2021-05-07 01:22:04,1390476997227212803,Harsh59680532
9,RT @ashwsbreal: 2000 #dogecoin to 5 people Whe...,2021-05-07 01:22:04,1390476994702311429,NenLb


In [6]:
tweets_df.to_csv('C:/Users/parth/OneDrive/Desktop/twitterfetcheddata.csv',encoding='utf-8')

<h3>Training and Testing the Given Dataset</h3>

In [7]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\spark\\spark-3.1.1-bin-hadoop2.7'

In [8]:
#import modules
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover
spark = SparkSession.builder.appName("TwitterSentimentAnalysis").getOrCreate()

In [9]:
import pandas as pd
tweets_csv = pd.read_csv('training_1.csv', names=['label','tweet_id','tweet_date_time','tweet_query','tweet_name','tweet_text'],encoding="ISO-8859-1")
tweets_csv.to_csv('training.csv')

In [10]:
#read csv file into dataFrame with automatically inferred schema
tweets_csv = spark.read.csv('training.csv', inferSchema=True, header=True)
tweets_csv.show(truncate=False, n=3)

+---+-----+----------+----------------------------+-----------+---------------+-------------------------------------------------------------------------------------------------------------------+
|_c0|label|tweet_id  |tweet_date_time             |tweet_query|tweet_name     |tweet_text                                                                                                         |
+---+-----+----------+----------------------------+-----------+---------------+-------------------------------------------------------------------------------------------------------------------+
|0  |0    |1467810369|Mon Apr 06 22:19:45 PDT 2009|NO_QUERY   |_TheSpecialOne_|@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D|
|1  |0    |1467810672|Mon Apr 06 22:19:49 PDT 2009|NO_QUERY   |scotthamilton  |is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!    |
|2  |0    |146781091

In [11]:
#select the related data
data = tweets_csv.select("label","tweet_text")

In [12]:
#Separate "SentimentText" into individual words using tokenizer
tokenizer = Tokenizer(inputCol="tweet_text", outputCol="SentimentWords")
tokenizedTrain = tokenizer.transform(data)
tokenizedTrain.show(truncate=False, n=3)

+-----+-------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------+
|label|tweet_text                                                                                                         |SentimentWords                                                                                                                          |
+-----+-------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------+
|0    |@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D|[@switchfoot, http://twitpic.com/2y1zl, -, awww,, that's, a, bummer., , you, shoulda, got, dav

In [13]:
#Removing stop words (unimportant words to be features)

swr = StopWordsRemover(inputCol=tokenizer.getOutputCol(), 
                       outputCol="MeaningfulWords")
SwRemovedTrain = swr.transform(tokenizedTrain)
SwRemovedTrain.show(truncate=False, n=3)

+-----+-------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------+
|label|tweet_text                                                                                                         |SentimentWords                                                                                                                          |MeaningfulWords                                                                                             |
+-----+-------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------+

In [14]:
#Converting words feature into numerical feature. In Spark 2.2.1,it is implemented in HashingTF funtion using Austin Appleby's MurmurHash 3 algorithm
hashTF = HashingTF(inputCol=swr.getOutputCol(), outputCol="features")
numericTrainData = hashTF.transform(SwRemovedTrain).select(
    'label', 'MeaningfulWords', 'features')
numericTrainData.show(truncate=False, n=3)

+-----+------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|MeaningfulWords                                                                                             |features                                                                                                                                                            |
+-----+------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0    |[@switchfoot, http://twitpic.com/2y1zl, -, awww,, bummer., , shoulda, got, david, carr, third, day, it., ;d]|(262144,[38640,52803,119493,127420,143

In [15]:
#Train our classifier model using training data
lr = LogisticRegression(labelCol="label", featuresCol="features", 
                        maxIter=10, regParam=0.3)
model = lr.fit(numericTrainData)
print ("Training is done!")

Training is done!


In [16]:
#Testing
tweets_test_csv = pd.read_csv('testdata_1.csv', names=['label','tweet_id','tweet_date_time','tweet_query','tweet_name','tweet_text'],encoding="ISO-8859-1")
tweets_test_csv.to_csv('testdata_1.csv')

In [17]:
#read csv file into dataFrame with automatically inferred schema
tweets_test_csv = spark.read.csv('testdata_1.csv', inferSchema=True, header=True)
tweets_test_csv.show(truncate=False, n=3)

+---+-----+--------+----------------------------+-----------+----------+---------------------------------------------------------------------------------------------------------------+
|_c0|label|tweet_id|tweet_date_time             |tweet_query|tweet_name|tweet_text                                                                                                     |
+---+-----+--------+----------------------------+-----------+----------+---------------------------------------------------------------------------------------------------------------+
|0  |4    |3       |Mon May 11 03:17:40 UTC 2009|kindle2    |tpryan    |@stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2 is fantastic in its own right.|
|1  |4    |4       |Mon May 11 03:18:03 UTC 2009|kindle2    |vcu451    |Reading my kindle2...  Love it... Lee childs is good read.                                                     |
|2  |4    |5       |Mon May 11 03:18:54 UTC 2009|kindle2    |chadfu    |Ok,

In [18]:
#select related testing data
test_data = tweets_test_csv.select("label","tweet_text")

In [19]:
#Prepare testing data
tokenizedTest = tokenizer.transform(test_data)
SwRemovedTest = swr.transform(tokenizedTest)
numericTest = hashTF.transform(SwRemovedTest).select(
    'Label', 'MeaningfulWords', 'features')
numericTest.show(truncate=False, n=2)

+-----+-----------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------+
|Label|MeaningfulWords                                                              |features                                                                                                     |
+-----+-----------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------+
|4    |[@stellargirl, loooooooovvvvvveee, kindle2., dx, cool,, 2, fantastic, right.]|(262144,[7877,12524,83462,122452,158475,167389,199176,258817],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])             |
|4    |[reading, kindle2..., , love, it..., lee, childs, good, read.]               |(262144,[32227,73008,113432,129422,186480,193997,198409,226565,249180],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
+-----+-------------

In [20]:
#Predict testing data and calculate the accuracy model
prediction = model.transform(numericTest)
predictionFinal = prediction.select(
    "MeaningfulWords", "prediction", "Label")
#predictionFinal.show(n=4, truncate = False)
correctPrediction = predictionFinal.filter(
    predictionFinal['prediction'] == predictionFinal['Label']).count()
totalData = predictionFinal.count()
print("accuracy:", correctPrediction/totalData)

accuracy: 0.8209704406023425


<h3>Predicting the Fetched Twitter Data</h3>

In [21]:
#read csv file into dataFrame with automatically inferred schema
fetched_csv = spark.read.csv('twitterfetcheddata.csv', inferSchema=True, header=True)
fetched_csv.show(truncate=False, n=3)

+---+--------------------------------------------------------------------------------------------------------------------------------------------+-------------------+-------------------+--------------+
|_c0|tweet_text                                                                                                                                  |tweet_datetime     |tweet_Id           |Twitter_name  |
+---+--------------------------------------------------------------------------------------------------------------------------------------------+-------------------+-------------------+--------------+
|0  |@Dogecoin_1USD @ashwsbreal @elonmusk DL9ut9CFbdQMSQ153NB1VAJietdcDmfZRB thanks!                                                             |2021-05-06 21:16:49|1390415277549133826|michel34901913|
|1  |RT @PnetworkC: we just get started @pNetworkDeFi #eidoo #cz_binance #Bitcoin #BNB #PNT #USDT #cryptocurrency  #DeFi #BSCPAD #BSC #Binance #…|2021-05-06 21:16:49|1390415277259739144|76Accu

In [22]:
#Predicting the fetched data
#select related fectched data
fetched_data = fetched_csv.select("tweet_text")

In [23]:
#Prepare testing data
tokenizedTest_fetch = tokenizer.transform(fetched_data)
SwRemovedTest_fetch = swr.transform(tokenizedTest_fetch)
numericTest_fetch = hashTF.transform(SwRemovedTest_fetch).select('tweet_text',
     'MeaningfulWords', 'features')
numericTest_fetch.show(truncate=False, n=2)

+--------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|tweet_text                                                                                                                                  |MeaningfulWords                                                                                                                                        |features                                                                                                                                                                                                |
+-------

In [24]:
#Predict fetched data
prediction = model.transform(numericTest_fetch)
predictionFinal_fetch = prediction.select("tweet_text","MeaningfulWords", "prediction")
predictionFinal_fetch.show(n=4, truncate = False)

+--------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------+----------+
|tweet_text                                                                                                                                  |MeaningfulWords                                                                                                                                         |prediction|
+--------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------+----------+
|@Dogecoin_1USD @ashwsbreal @elonmusk DL9ut9CFbdQMSQ153NB1VAJietdcDmfZRB thanks

<h3>Save the Fetched data and Classification Result Into SQL Database</h3>

In [25]:
#save the fetched data and classification result into SQL Database
#set variable to be used to connect the database
database = "TwitterDB"
table = "dbo.tbl_spark_df"
user = "user"
password  = ""
 
#write the dataframe into a sql table
predictionFinal_fetch.write.mode("overwrite") \
    .format("jdbc") \
    .option("url", f"jdbc:sqlserver://localhost:1433;databaseName={database};") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .save()